### This script aims to retrieve economic parameter information from the World Bank data.

In [1]:
# Importing libraries
import requests
import pandas as pd
import useful_functions
from datetime import datetime, timedelta

In [10]:
# Define the last month to get data

end_period = useful_functions.define_end_period('%Y-%m')


In [11]:
###Source: https://medium.com/@akshaybagal/oecd-stats-website-your-go-to-for-comprehensive-statistics-datasets-on-oecd-countries-2ae04e4aa044
#THIS IS SCRIPT WAS CREATED TO GET INFLATION FROM OECD WEBSITE


# URL 
#Consumer price indices (CPIs) - Complete database  : MT - Annual inflation - 2001-2023
#url = 'https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/CAN+CHL+GBR+USA+EA20+ARG+CHN.CPALTT01.GP.M/all?startTime=2001-01&endTime=2023-12&dimensionAtObservation=allDimensions'
url = f'https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/CAN+CHL+GBR+USA+EA20+ARG+CHN.CPALTT01.GP.M/all?startTime=2001-01&endTime={end_period}&dimensionAtObservation=allDimensions'


response = requests.request("GET", url)

data = response.json()
data_values = data['dataSets'][0]['observations']
dimensions = data['structure']['dimensions']['observation']

id_to_name_mappings = {
    dim['name']: {item['id']: item['name'] for item in dim['values']}
    for dim in dimensions
}

dimension_values = [dim['values'] for dim in dimensions]

def get_id_from_index(dim_index, index):
    return dimension_values[dim_index][index]['id']

def map_id_to_name(dim_name, id):
    return id_to_name_mappings[dim_name].get(id, id)

rows = []
for key, value in data_values.items():
    indices = key.split(':')  # Split keys into separate dimension indices
    country = map_id_to_name('Country', get_id_from_index(0, int(indices[0])))
    subject = map_id_to_name('Subject', get_id_from_index(1, int(indices[1])))
    measure = map_id_to_name('Measure', get_id_from_index(2, int(indices[2])))  # Corrected typo here
    frequency = map_id_to_name('Frequency', get_id_from_index(3, int(indices[3])))
    time = map_id_to_name('Time', get_id_from_index(4, int(indices[4])))
    data_value = value[0]  # Extract the data value
    rows.append([country, subject, measure, frequency, time, data_value])  # Append the row to the list of rows

df_inflation = pd.DataFrame(rows, columns=['Country', 'Subject', 'Measure', 'Frequency', 'Time', 'Value'])

df_inflation['Time'] = pd.to_datetime(df_inflation['Time'], format='%b-%Y').dt.strftime('%Y-%m-%d')

df_inflation_rotated = df_inflation.pivot(index='Time', columns=['Country'], values='Value')

df_inflation_rotated = df_inflation_rotated.rename(columns={
    'Argentina': 'ARG_inflation',
    'Canada': 'CAN_inflation',
    'Chile': 'CHI_inflation',
    "China (People's Republic of)": 'CHN_inflation',
    'Euro area (20 countries)': 'EUZ_inflation',
    'United Kingdom': 'UK_inflation',
    'United States': 'US_inflation'
})




In [12]:
# Adjust index date format
#df_inflation_rotated = useful_functions.adjust_index_date_format(df_inflation_rotated)

# Convert the dataset to monthly frequency getting the last valid value of each month
df_inflation_rotated_monthly = useful_functions.convert_df_to_monthly(df_inflation_rotated,'%Y-%m-%d')

#Export to csv
df_inflation_rotated_monthly.to_csv('../data/df_inflation.csv')

In [6]:
###Source: https://medium.com/@akshaybagal/oecd-stats-website-your-go-to-for-comprehensive-statistics-datasets-on-oecd-countries-2ae04e4aa044

#THIS IS SCRIPT WAS CREATED TO GET INTEREST RATES FROM OECD WEBSITE

import requests

import pandas as pd


# OECD - Monthly Monetary and Financial Statistics (MEI)  : Interest rates
#url = 'https://stats.oecd.org/SDMX-JSON/data/MEI_FIN/IRLT+IR3TIB+IRSTCI.CAN+CHL+GBR+USA+EA19+ARG+CHN.M/all?startTime=2001-01&endTime=2023-12&dimensionAtObservation=allDimensions'
url = f'https://stats.oecd.org/SDMX-JSON/data/MEI_FIN/IRSTCI.CAN+CHL+GBR+USA+EA19+ARG+CHN.M/all?startTime=2001-01&endTime{end_period}2&dimensionAtObservation=allDimensions'

response = requests.request("GET", url)

data = response.json()
data_values = data['dataSets'][0]['observations']
dimensions = data['structure']['dimensions']['observation']

id_to_name_mappings = {
    dim['name']: {item['id']: item['name'] for item in dim['values']}
    for dim in dimensions
}

dimension_values = [dim['values'] for dim in dimensions]

def get_id_from_index(dim_index, index):
    return dimension_values[dim_index][index]['id']

def map_id_to_name(dim_name, id):
    return id_to_name_mappings[dim_name].get(id, id)

rows = []
for key, value in data_values.items():
    indices = key.split(':')  # Split keys into separate dimension indices
    subject = map_id_to_name('Subject', get_id_from_index(0, int(indices[0])))
    country = map_id_to_name('Country', get_id_from_index(1, int(indices[1])))
    frequency = map_id_to_name('Frequency', get_id_from_index(2, int(indices[2])))
    time = map_id_to_name('Time', get_id_from_index(3, int(indices[3])))
    data_value = value[0]  # Extract the data value
    rows.append([country, subject, frequency, time, data_value])  # Append the row to the list of rows


df_interest = pd.DataFrame(rows, columns=['Country', 'Subject', 'Frequency', 'Time', 'Value'])


df_interest['Time'] = pd.to_datetime(df_interest['Time'], format='%b-%Y').dt.strftime('%Y-%m-%d')
#df = df[df['Subject'].str.contains('Short-term interest rates')]
df_interest = df_interest[df_interest['Subject'].str.contains('Immediate')]

df_interest.set_index('Time', inplace=True)
df_interest.sort_index(inplace=True)

df_interest_rotated = df_interest.pivot(columns=['Country'], values='Value')


df_interest_rotated = df_interest_rotated.rename(columns={
    'Canada': 'CAN_interest',
    'Chile': 'CHI_interest',
    "China (People's Republic of)": 'CHN_interest',
    'Euro area (20 countries)': 'EUZ_interest',
    'United Kingdom': 'UK_interest',
    'United States': 'US_interest'
})




In [7]:
# Adjust index date format
#df_interest_rotated = useful_functions.adjust_index_date_format(df_interest_rotated)

# Convert the dataset to monthly frequency getting the last valid value of each month
df_interest_rotated_monthly = useful_functions.convert_df_to_monthly(df_interest_rotated,'%Y-%m-%d')

df_interest_rotated_monthly.to_csv('../data/df_interest_rate.csv')

In [8]:
#Get the GDP
'''
Key short-term economic indicators
The Key Economic Indicators (KEI) database contains monthly and quarterly statistics (and associated statistical methodological information) for all OECD member countries and for a selection of non-member countries on a wide variety of economic indicators, namely: quarterly national accounts, industrial production, composite leading indicators, business tendency and consumer opinion surveys, retail trade, consumer and producer prices, hourly earnings, employment/unemployment, interest rates, monetary aggregates, exchange rates, international trade and balance of payments.
Indicators have been prepared by national statistical agencies primarily to meet the requirements of users within their own country. In most instances, the indicators are compiled in accordance with international statistical guidelines and recommendations. However, national practices may depart from these guidelines, and these departures may impact on comparability between countries. There is an on-going process of review and revision of the contents of the database in order to maximise the relevance of the database for short-term economic analysis.
Reference series (GDP) - Ratio to trend
Available at https://data-explorer.oecd.org/vis?tm=monthly%20gdp&pg=0&hc[Frequency%20of%20observation]=Monthly&snb=6&vw=ov&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_KEI%40DF_KEI&df[ag]=OECD.SDD.STES&df[vs]=4.0&pd=2001-01%2C2023-12&dq=CHN%2BUSA%2BGBR.M.RS.IX._T..&ly[rw]=REF_AREA&ly[cl]=TIME_PERIOD&to[TIME_PERIOD]=false

'''

url = f'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_KEI@DF_KEI,4.0/CHN+USA+GBR.M.RS.IX._T..?startPeriod=2001-01&endPeriod={end_period}&dimensionAtObservation=AllDimensions'
response = requests.request("GET", url)


import xml.etree.ElementTree as ET
import pandas as pd

# Your XML data (replace with the actual XML string)
xml_data = response.text

# Parse the XML data
root = ET.fromstring(xml_data)

# Define a function to extract data from each observation
def extract_observation(observation):
    obs_key = observation.find('.//generic:ObsKey', namespaces=NAMESPACES)
    obs_value = observation.find('.//generic:ObsValue', namespaces=NAMESPACES)
    attributes = observation.findall('.//generic:Attributes/generic:Value', namespaces=NAMESPACES)

    data = {
        'TIME_PERIOD': obs_key.find('.//generic:Value[@id="TIME_PERIOD"]', namespaces=NAMESPACES).get('value'),
        'COUNTRY': obs_key.find('.//generic:Value[@id="REF_AREA"]', namespaces=NAMESPACES).get('value'),
        'VALUE': obs_value.get('value'),
    }

    for attr in attributes:
        data[attr.get('id')] = attr.get('value')

    return data

# Define XML namespaces
NAMESPACES = {
    'generic': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic',
}

# Extract observations from the XML
observations = root.findall('.//generic:Obs', namespaces=NAMESPACES)

# Convert data to a list of dictionaries
data_list = [extract_observation(observation) for observation in observations]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)




df_gdp_raw = df[['TIME_PERIOD', 'COUNTRY', 'VALUE']].copy()


df_gdp_raw['TIME_PERIOD'] = pd.to_datetime(df_gdp_raw['TIME_PERIOD'], format='%Y-%m').dt.strftime('%Y-%m-%d')

df_gdp_raw.rename(columns={'TIME_PERIOD': 'Time', 'COUNTRY': 'country', 'VALUE': 'value'}, inplace=True)

df_gdp_raw.set_index('Time', inplace=True)


df_gdp_rotated = df_gdp_raw.pivot(columns='country', values='value')


df_gdp = df_gdp_rotated.rename(columns={
    "CHN": 'CHN_GDP',
    'GBR': 'UK_GDP',
    'USA': 'US_GDP'
})




In [9]:
# Convert the dataset to monthly frequency getting the last valid value of each month
df_gdp_monthly = useful_functions.convert_df_to_monthly(df_gdp,'%Y-%m-%d')

# Adjust index date format
#df_gdp = useful_functions.adjust_index_date_format(df_gdp)

df_gdp_monthly.to_csv('../data/df_gdp_rate.csv')